In [1]:
### sakila database..

# 월별 렌탈 횟수가 30회 이상인 유저 이름과 이메일을 출력..
# 제일 많은 매출을 발생시킨 영화제목..
# file 테이블에서 2006년이나 2007년에 출시되었으면서, PG등급이거나 G등급의 영화제목
# file 테이블에 설명과 'boring'이 포함되면서 렌탈비용이 0.99인 영화제목, 설명, 렌탈 비용

In [3]:
import pandas as pd
import MySQLdb

In [6]:
db = MySQLdb.connect(host='localhost', user='root', password='1234')
cursor = db.cursor()

In [8]:
sql = 'show databases'

cursor.execute(sql)
cursor.fetchall()

(('information_schema',),
 ('budongsan',),
 ('employees',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('sys',),
 ('tabledb',),
 ('world',))

In [9]:
sql = 'use sakila'

cursor.execute(sql)

0

In [12]:
sql = 'show tables'

cursor.execute(sql)
cursor.fetchall()

(('actor',),
 ('actor_info',),
 ('address',),
 ('category',),
 ('city',),
 ('country',),
 ('customer',),
 ('customer_list',),
 ('film',),
 ('film_actor',),
 ('film_category',),
 ('film_list',),
 ('film_text',),
 ('inventory',),
 ('language',),
 ('nicer_but_slower_film_list',),
 ('payment',),
 ('rental',),
 ('sales_by_film_category',),
 ('sales_by_store',),
 ('staff',),
 ('staff_list',),
 ('store',))

In [63]:
sql_customer = """
select * from customer;
"""

df_customer = pd.read_sql(sql_customer, db)
df_customer

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20
2,3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-14 22:04:36,2006-02-15 04:57:20
3,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-14 22:04:36,2006-02-15 04:57:20
4,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20
...,...,...,...,...,...,...,...,...,...
594,595,1,TERRENCE,GUNDERSON,TERRENCE.GUNDERSON@sakilacustomer.org,601,1,2006-02-14 22:04:37,2006-02-15 04:57:20
595,596,1,ENRIQUE,FORSYTHE,ENRIQUE.FORSYTHE@sakilacustomer.org,602,1,2006-02-14 22:04:37,2006-02-15 04:57:20
596,597,1,FREDDIE,DUGGAN,FREDDIE.DUGGAN@sakilacustomer.org,603,1,2006-02-14 22:04:37,2006-02-15 04:57:20
597,598,1,WADE,DELVALLE,WADE.DELVALLE@sakilacustomer.org,604,1,2006-02-14 22:04:37,2006-02-15 04:57:20


In [118]:
sql_rental = """
select * from rental;
"""

sql_customer = """
select * from customer;
"""

df_rental = pd.read_sql(sql_rental, db)
df_customer = pd.read_sql(sql_customer, db)
df = pd.merge(df_rental, df_customer, on='customer_id')
df_group = df.groupby('customer_id')['rental_id'].count()
df[df['customer_id'].isin(df_group[df_group >= 30].index.tolist())].drop_duplicates(['customer_id']).loc[:, ['first_name', 'last_name', 'email']]


,first_name,last_name,email
24,TOMMY,COLLAZO,TOMMY.COLLAZO@sakilacustomer.org
62,MANUEL,MURRELL,MANUEL.MURRELL@sakilacustomer.org
160,CASSANDRA,WALTERS,CASSANDRA.WALTERS@sakilacustomer.org
190,MINNIE,ROMERO,MINNIE.ROMERO@sakilacustomer.org
325,RAYMOND,MCWHORTER,RAYMOND.MCWHORTER@sakilacustomer.org
...,...,...,...
15517,HAROLD,MARTINO,HAROLD.MARTINO@sakilacustomer.org
15574,LORI,WOOD,LORI.WOOD@sakilacustomer.org
15683,HELEN,HARRIS,HELEN.HARRIS@sakilacustomer.org
15794,MARILYN,ROSS,MARILYN.ROSS@sakilacustomer.org


In [104]:
df['customer_id'].sort_values()

2006       1
2010       1
2009       1
2008       1
2007       1
        ... 
13251    599
13250    599
13268    599
13258    599
13267    599
Name: customer_id, Length: 16044, dtype: int64